In [1]:
from cuml import RandomForestClassifier as cuRF
from sklearn.ensemble import RandomForestClassifier as sklRF
from sklearn.metrics import accuracy_score
import cudf
import numpy as np
import pandas as pd

In [2]:
col_names = ['label'] + ["col-{}".format(i) for i in range(2, 30)]
dtypes_ls = ['int32'] + ['float32' for _ in range(2, 30)]
data = cudf.read_csv('/datasets/higgs/HIGGS.csv', names=col_names, dtype=dtypes_ls)
data.head().to_pandas()

,label,col-2,col-3,col-4,col-5,col-6,col-7,col-8,col-9,col-10,...,col-20,col-21,col-22,col-23,col-24,col-25,col-26,col-27,col-28,col-29
0,1,0.869293,-0.635082,0.225690,0.327470,-0.689993,0.754202,-0.248573,-1.092064,0.000000,...,-0.010455,-0.045767,3.101961,1.353760,0.979563,0.978076,0.920005,0.721657,0.988751,0.876678
1,1,0.907542,0.329147,0.359412,1.497970,-0.313010,1.095531,-0.557525,-1.588230,2.173076,...,-1.138930,-0.000819,0.000000,0.302220,0.833048,0.985700,0.978098,0.779732,0.992356,0.798343
2,1,0.798835,1.470639,-1.635975,0.453773,0.425629,1.104875,1.282323,1.381664,0.000000,...,1.128848,0.900461,0.000000,0.909753,1.108331,0.985692,0.951331,0.803251,0.865924,0.780118
3,0,1.344385,-0.876626,0.935913,1.992050,0.882454,1.786066,-1.646778,-0.942383,0.000000,...,-0.678379,-1.360356,0.000000,0.946652,1.028704,0.998656,0.728281,0.869200,1.026736,0.957904
4,1,1.105009,0.321356,1.522401,0.882808,-1.205349,0.681466,-1.070464,-0.921871,0.000000,...,-0.373566,0.113041,0.000000,0.755856,1.361057,0.986610,0.838085,1.133295,0.872245,0.808486


In [3]:
test_rows = 500000
train, test = data[:-test_rows], data[-test_rows:]
del data

X_train, y_train = train[train.columns.difference(['label'])], train['label']
X_test, y_test = test[test.columns.difference(['label'])], test['label']
del train, test

In [4]:
X_train, y_train = X_train.as_matrix(), y_train.to_array()
X_test, y_test = X_test.as_matrix(), y_test.to_array()

In [5]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(10500000, 28) (10500000,) (500000, 28) (500000,)


In [6]:
cu_rf_params = {
    'n_estimators': 100,
    'max_depth': 25,
    'n_bins': 4,
}

In [7]:
%%time
cu_rf = cuRF(**cu_rf_params)
cu_rf.fit(X_train, y_train)

CPU times: user 2h 9min 59s, sys: 36min 26s, total: 2h 46min 26s
Wall time: 2h 46min 22s


In [8]:
accuracy_score(cu_rf.predict(X_test), y_test)

0.501516

In [ ]:
skl_rf_params = {
    'n_estimators': 50,
    'max_depth': 12,
}

In [ ]:
%%time
skl_rf = sklRF(**skl_rf_params, n_jobs=-1)
skl_rf.fit(X_train, y_train)

In [ ]:
accuracy_score(skl_rf.predict(X_test), y_test)